## Deliverable 2. Create a Customer Travel Destinations Map.

In [13]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [14]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vaini,TO,-21.2000,-175.2000,82.56,83,40,10.36,scattered clouds
1,1,Atuona,PF,-9.8000,-139.0333,78.39,71,100,18.48,overcast clouds
2,2,Tay Ninh,VN,11.3000,106.1000,77.40,87,100,3.02,overcast clouds
3,3,Mataura,NZ,-46.1927,168.8643,57.25,59,65,6.67,broken clouds
4,4,Butaritari,KI,3.0707,172.7902,81.70,72,31,12.26,scattered clouds


In [18]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [19]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vaini,TO,-21.2000,-175.2000,82.56,83,40,10.36,scattered clouds
1,1,Atuona,PF,-9.8000,-139.0333,78.39,71,100,18.48,overcast clouds
2,2,Tay Ninh,VN,11.3000,106.1000,77.40,87,100,3.02,overcast clouds
4,4,Butaritari,KI,3.0707,172.7902,81.70,72,31,12.26,scattered clouds
17,17,Rikitea,PF,-23.1203,-134.9692,75.58,76,16,13.98,light rain
25,25,Yanam,IN,16.7333,82.2167,81.00,95,100,1.90,overcast clouds
26,26,Alofi,NU,-19.0595,-169.9187,77.27,78,25,20.29,light rain
29,29,Bullhead City,US,35.1478,-114.5683,86.29,10,1,16.11,clear sky
34,34,Kapaa,US,22.0752,-159.3190,82.38,78,1,5.01,moderate rain
43,43,Acapulco,MX,16.8634,-99.8901,81.21,84,54,6.62,broken clouds


In [20]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City ID                203
City                   203
Country                203
Lat                    203
Lng                    203
Max Temp               203
Humidity               203
Cloudiness             203
Wind Speed             203
Current Description    203
dtype: int64

In [21]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna() 

In [22]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vaini,TO,82.56,scattered clouds,-21.2000,-175.2000,
1,Atuona,PF,78.39,overcast clouds,-9.8000,-139.0333,
2,Tay Ninh,VN,77.40,overcast clouds,11.3000,106.1000,
4,Butaritari,KI,81.70,scattered clouds,3.0707,172.7902,
17,Rikitea,PF,75.58,light rain,-23.1203,-134.9692,
25,Yanam,IN,81.00,overcast clouds,16.7333,82.2167,
26,Alofi,NU,77.27,light rain,-19.0595,-169.9187,
29,Bullhead City,US,86.29,clear sky,35.1478,-114.5683,
34,Kapaa,US,82.38,moderate rain,22.0752,-159.3190,
43,Acapulco,MX,81.21,broken clouds,16.8634,-99.8901,


In [24]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params['location'] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [30]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vaini,TO,82.56,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
1,Atuona,PF,78.39,overcast clouds,-9.8000,-139.0333,Villa Enata
2,Tay Ninh,VN,77.40,overcast clouds,11.3000,106.1000,Thanh Tuyền Hotel
4,Butaritari,KI,81.70,scattered clouds,3.0707,172.7902,Isles Sunset Lodge
17,Rikitea,PF,75.58,light rain,-23.1203,-134.9692,People ThankYou
25,Yanam,IN,81.00,overcast clouds,16.7333,82.2167,The Regency Hotel
26,Alofi,NU,77.27,light rain,-19.0595,-169.9187,Taloa Heights
29,Bullhead City,US,86.29,clear sky,35.1478,-114.5683,Harrah's Laughlin
34,Kapaa,US,82.38,moderate rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
43,Acapulco,MX,81.21,broken clouds,16.8634,-99.8901,HS Hotsson Smart Acapulco


In [37]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [33]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [34]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))